# Unicode code points

Unicode code points are assigning an integer to every character and script across different writing systems. 

In [4]:
ord("N") #using the ord function in python we can display the unicode encoding of a character

78

We cannot use the unicode code points standard as our vocabulary for a LLM. First of all because we would end up with a gigantic vocabulary size but also because unicode is not a stable representation of the characters as it keeps on being updated. 

# UTF-8

One encoding standard for representing every character in the unicode standard is UTF-8 which uses 1 to 4 bytes per character. The first 128 character of UTF-8 are the same than ASCII making UTF-8 backward compatible with ASCII. 


In [5]:
list("Hello World!".encode("utf-8"))

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33]

We cannot keep this as our vocabulary because we only a certain context windown size for our transformer and keeping utf_! just like so would make us end up with very long sequences adn therefore end up with an attention becoming extremely expensive. That's why we will use the byte pair encoding algorithm to leverage compression.